# Solving SAT problem with Grover's algorithm Kata Workbook

**What is this workbook?**
A workbook is a collection of problems, accompanied by solutions to them. 
The explanations focus on the logical steps required to solve a problem; they illustrate the concepts that need to be applied to come up with a solution to the problem, explaining the mathematical steps required. 

Note that a workbook should not be the primary source of knowledge on the subject matter; it assumes that you've already read a tutorial or a textbook and that you are now seeking to improve your problem-solving skills. You should attempt solving the tasks of the respective kata first, and turn to the workbook only if stuck. While a textbook emphasizes knowledge acquisition, a workbook emphasizes skill acquisition.

This workbook describes the solutions to the problems offered in the [Solving SAT problem with Grover's algorithm kata](./SolveSATWithGrover.ipynb). Since the tasks are offered as programming problems, the explanations also cover some elements of Q# that might be non-obvious for a first-time user.

**What you should know for this workbook**

You should be familiar with the following concepts before tackling the Distinguish Unitaries kata (and this workbook):

1. [Basic linear algebra](./../tutorials/LinearAlgebra/LinearAlgebra.ipynb)).
2. [Single-qubit](./../tutorials/SingleQubitGates/SingleQubitGates.ipynb) and [multi-qubit quantum gates](./../tutorials/MultiQubitGates/MultiQubitGates.ipynb) and using them to manipulate the state of the system.
3. [Grover's Algorithm](./../tutorials/ExploringGroversAlgorithm/ExploringGroversAlgorithm.ipynb) and its [Implementation](./../GroversAlgorithm/GroversAlgorithm.ipynb).
4. Basic Knowledge of Boolean Gates and 3SAT Problem.



To begin, first prepare this notebook for execution (if you skip this step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells):

In [ ]:
%package Microsoft.Quantum.Katas::0.12.20072031

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


In [ ]:
%workspace reload

## Part I. Oracles for SAT problems

### Task 1.1. The AND oracle: $f(x) = x_0 \wedge x_1$

**Inputs:** 

  1. 2 qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
i.e., flip the target state if all qubits of the query register are in the $|1\rangle$ state, 
and leave it unchanged otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

### Solution

The goal is to flip the qubit $|y\rangle$ if and only if all qubits in the register $|x\rangle$ are in the state $|1\rangle$.  
Hence the required unitary $U_{and}$ is such that:
$$U_{and}|x\rangle|y\rangle = \begin{cases} 
          |x\rangle|y\rangle & \text{if }x \neq 1...1 \\
          |x\rangle X|y\rangle & \text{if }x = 1...1 
       \end{cases}$$
       
We can rewrite this as 

$$U_{and} = \big(I - |1...1\rangle\langle 1...1|\big) \otimes I + |1...1\rangle\langle 1...1|\otimes X$$ 

Here we're using a convenient shorthand for ket-bra notation of gate effect on multiple basis states at once:

$$I - |1...1\rangle\langle 1...1| = \sum_{i \in \{0,1\}^n,\\ |i\rangle\neq |1...1\rangle}|i\rangle\langle i|$$

For the number of qubits in the input register $n=2$, $U_{and}$ is the **Toffoli** gate:
$$U_{and} = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
\end{bmatrix}$$

In the general case of arbitrary number of qubits in the input, 
this can be represented as a `Controlled X` gate, with the input register $|x\rangle$ as control and the target qubit $|y\rangle$ as target. 

> In Q# you can use [Controlled functor](https://docs.microsoft.com/quantum/user-guide/using-qsharp/operations-functions#controlled-and-adjoint-operations) to apply a controlled version of the gate, if the gate specifies it.
>
> If $|x\rangle$ consists of just 2 qubits, we could also use the built-in **Toffoli** gate - the [`CCNOT` gate](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic.ccnot).

In [ ]:
%kata T11_Oracle_And_Test 

operation Oracle_And (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    Controlled X(queryRegister, target); 
}

[Return to Task 1.1 of the Solving SAT with Grover's Kata.](./SolveSATWithGrover.ipynb#Task-1.1.-The-AND-oracle:-$f(x)-=-x_0-\wedge-x_1$)

### Task 1.2. The OR oracle: $f(x) = x_0 \vee x_1$

**Inputs:** 

  1. 2 qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:** 

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
i.e., flip the target state if at least one qubit of the query register is in the $|1\rangle$ state, 
and leave it unchanged otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

### Solution

The goal is to flip the qubit $|y\rangle$ if at least one qubit in the register $|x\rangle$ is in the state $|1\rangle$.  
This can also be restated as to flip the qubit $|y\rangle$ *unless* the register $|x\rangle$ is in the state $|0...0\rangle$.

Hence the required unitary $U_{or}$ is such that:
$$U_{or}|x\rangle|y\rangle = \begin{cases} 
          |x\rangle |y\rangle & \text{if }x = 0...0  \\
          |x\rangle X|y\rangle & \text{if }x \neq 0...0
       \end{cases}$$

We can rewrite this as
$$U_{or} = |0...0\rangle\langle 0...0|\otimes I + \big(I - |0...0\rangle\langle 0...0|\big) \otimes X$$  

We can now reduce this problem to the AND oracle using the fact that OR gate can be created using an AND gate and NOT gates.  

$$x_0\vee x_1\vee ... \vee x_{n-1} = \lnot(\lnot x_0\wedge \lnot x_1\wedge ... \wedge \lnot x_{n-1})$$

Thus $U_{or}$ can be rewritten as $U_{or}=(X^{\otimes n}\otimes X) \cdot U_{and} \cdot (X^{\otimes n}\otimes I)$. 

> <details>
    <summary><b>Click here to see how to get this matrix expression using ket-bra notation</b></summary>
>    
$$
U_{or}= (X^{\otimes n}\otimes X) \cdot U_{and} \cdot (X^{\otimes n}\otimes I) = \\
 = (X^{\otimes n}\otimes X) \cdot (\sum_{i \in \{0,1\}^n,\\ |i\rangle\neq |1...1\rangle}|i\rangle\langle i| \otimes I + |1...1\rangle\langle 1...1|\otimes X) \cdot (X^{\otimes n}\otimes I) = \\
 = (X^{\otimes n}\otimes X) \cdot (\sum_{i \in \{0,1\}^n,\\ |i\rangle\neq |1...1\rangle}|i\rangle\langle i|X^{\otimes n} \otimes I + |1...1\rangle\langle 1...1|X^{\otimes n}\otimes X) = \\
 = (X^{\otimes n}\otimes X) \cdot (\sum_{i \in \{0,1\}^n,\\ |i\rangle\neq |1...1\rangle}|i\rangle\langle i|X^{\otimes n} \otimes I + |1...1\rangle\langle 0...0|\otimes X) = \\
 = \sum_{i \in \{0,1\}^n,\\ |i\rangle\neq |1...1\rangle}X^{\otimes n}|i\rangle\langle i|X^{\otimes n} \otimes X + X^{\otimes n}|1...1\rangle\langle 0...0|\otimes I = \\
 = \sum_{i \in \{0,1\}^n,\\ |i\rangle\neq |1...1\rangle}X^{\otimes n}|i\rangle\langle i|X^{\otimes n} \otimes X + |0...0\rangle\langle 0...0|\otimes I = \\
 = \sum_{i \in \{0,1\}^n,\\ |i\rangle\neq |0...0\rangle}|i\rangle\langle i| \otimes X + |0...0\rangle\langle 0...0|\otimes I$$
</details>

This corresponds to the following steps:

1. Flip all the qubits of $|x\rangle$ using X gates to convert the inputs from $|x_0, x_1, ..., x_{n-1}\rangle$ to $|\lnot x_0, \lnot x_1, ..., \lnot x_{n-1}\rangle$.
2. Apply $U_{and}$ oracle to convert $|y\rangle$ to $|y \oplus (\lnot x_0\wedge \lnot x_1\wedge ... \wedge \lnot x_{n-1}) \rangle$.
3. Undo the flipping to return the inputs to $|x_0, x_1, ..., x_{n-1}\rangle$.
4. Flip the state of the target qubit $|y\rangle$ again to perform the final negation $|y \oplus \lnot(\lnot x_0\wedge \lnot x_1\wedge ... \wedge \lnot x_{n-1}) \rangle = |y \oplus x_0\vee x_1\vee ... \vee x_{n-1} \rangle$.

> We can use the ` within ... apply ...` construct in Q# implement the pattern of modifying the qubits of $|x\rangle$, applying the oracle to get the result and write it into $|y\rangle$ and undoing (uncomputing) the modification to $|x\rangle$.

In [ ]:
%kata T12_Oracle_Or_Test 

operation Oracle_Or (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    within {
        // Flip input qubits to negate them
        ApplyToEachA(X, queryRegister);
    } apply {
        // Use the AND oracle to get the AND of negated inputs
        Oracle_And(queryRegister, target);
    }
    // Flip the target to get the final answer
    X(target);
}

[Return to Task 1.2 of the Solving SAT with Grover's Kata.](./SolveSATWithGrover.ipynb#Task-1.2.-The-OR-oracle:-$f(x)-=-x_0-\vee-x_1$)

### Task 1.3. The XOR oracle: $f(x) = x_0 \oplus x_1$

**Inputs:** 

  1. 2 qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:** 

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
i.e., flip the target state if the qubits of the query register are in different states, 
and leave it unchanged otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

### Solution

Let's consider the case of $n$-qubit register $|x\rangle$; then $f(x) = x_0 \oplus x_1 \oplus ... \oplus x_{n-1}$. 
The goal is to flip the qubit $|y\rangle$ if and only if $f(x) = 1$.  

Hence the effect of the required unitary $U_{xor}$ on the basis state $|x\rangle|y\rangle$ will be:

$$U_{xor}|x\rangle|y\rangle = |x\rangle|y\oplus f(x)\rangle = \\
= |x\rangle|y\oplus x_0 \oplus x_1 \oplus ... \oplus x_{n-1}\rangle = \\
= |x\rangle|(...((y\oplus x_0) \oplus x_1) ... )\oplus x_{n-1}\rangle$$

Let's consider a unitary $U_{\oplus i}$, which has the following effect on the basis state:

$$U_{\oplus i}|x\rangle|y\rangle = |x\rangle|y \oplus x_i\rangle$$

This unitary is simply a CNOT gate with i-th qubit $|x_i\rangle$ as control and $|y\rangle$ as target.

Then we can rewrite the effect of our unitary $U_{xor}$ as follows:

$$U_{xor}|x\rangle|y\rangle = U_{\oplus n-1} (U_{\oplus n-2} (... U_{\oplus 0} |x\rangle|y\rangle ...)) = \prod_{i=0}^{n-1}U_{\oplus i}|x\rangle|y\rangle$$

We see that we can write

$$U_{xor}=\prod_{i=0}^{n-1}U_{\oplus i}$$

We can implement this as a sequence of CNOT gates, with each of the qubits of the input register as controls and the output qubit as target.

In [ ]:
%kata T13_Oracle_Xor_Test 

operation Oracle_Xor (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    ApplyToEachA(CNOT(_, target), queryRegister); 
}

[Return to Task 1.3 of the Solving SAT with Grover's Kata.](./SolveSATWithGrover.ipynb#Task-1.3.-The-XOR-oracle:-$f(x)-=-x_0-\oplus-x_1$)

### Task 1.4. Alternating bits oracle: $f(x) = (x_0 \oplus x_1) \wedge (x_1 \oplus x_2) \wedge \dots \wedge (x_{N-2} \oplus x_{N-1})$

**Inputs:** 

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

### Solution

Let $x_i'=x_i\oplus x_{i+1}$, then we can rewrite $f(x)$ as $f(x) = x_0' \wedge x_1' \wedge ... \wedge x_{n-2}'$.  
We can use $U_{xor}$ or `Oracle_Xor` from the previous task since $U_{xor}|x_l\rangle|x_r\rangle|0\rangle = |x_l\rangle|x_r\rangle|x_l \oplus x_r\rangle$.
Thus we can take the XOR of all adjacent qubits and store the result in an `ancillaryRegister`(This procedure can be undone in the same way). 

Now we can find $f(x)$ i.e transform $|y\rangle$ into $|y \oplus f(x)\rangle$ using the `Oracle_And` on `ancillaryRegister` and `target`.

Hence the final solution consists of:
1. Finding XOR of all adjancent qubits and storing them in an ancillary register.
2. `apply`ing $U_{and}$ or `Oracle_And` on `ancillaryRegister`.
3. Undo the computation done in 1. to reset the `ancillaryRegister`.


In [ ]:
%kata T14_Oracle_AlternatingBits_Test 

open Microsoft.Quantum.Arrays;

operation Oracle_AlternatingBits (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    let N = Length(queryRegister);
    using(ancillaryRegister = Qubit[N-1]){
        // Find the XOR of all consecutive qubits and store the result in ancillaryRegister
        within {
            for(i in 0 .. N-2){
                Oracle_Xor(queryRegister[i..i+1],ancillaryRegister[i]);
            }
        }
        // Use the And Oracle on ancillaryRegister to get the answer.
        apply {
            Oracle_And(ancillaryRegister,target); 
        }
    }
    
}

We can also avoid using the `ancillaryRegister` by storing the result of `XOR` operation in the `queryRegister` itself. 
This would simply be $CNOT$ of two adjacent qubits with the target qubit being the one in which we store the result. 
Since all qubits except those at index $0$ and $N-1$ are inputs to 2 $XOR$ operations, the order of these operations must be carefully chosen.
If going from index $0$ to $N-1$ the target must always be the lower index i.e.  `CNOT(queryRegister[i+1],queryRegister[i])`. This way `queryRegister[i+1]` is preserved for the next $XOR$ operation.

Now the `queryRegister[0..N-2]` contains the $XOR$ results and perform the `Oracle_And` operation. 

In [ ]:
open Microsoft.Quantum.Arrays;

operation Oracle_AlternatingBits_2 (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    // Find the XOR of all consecutive qubits and store the result in queryRegister itself.
    within {
        for(i in 0 .. N-2){
            CNOT(queryRegister[i+1],queryRegister[i]);
        }
    }
    // Use the And Oracle on First N-1 Qubits of queryRegister to get the answer.
    apply {
        Oracle_And(Most(queryRegister),target); 
    }
}

[Return to Task 1.4 of the Solving SAT with Grover's Kata.](./SolveSATWithGrover.ipynb#Task-1.4.-Alternating-bits-oracle:-$f(x)-=-(x_0-\oplus-x_1)-\wedge-(x_1-\oplus-x_2)-\wedge-\dots-\wedge-(x_{N-2}-\oplus-x_{N-1})$)

### Task 1.5. Evaluate one clause of a SAT formula

> For SAT problems, $f(x)$ is represented as a conjunction (an AND operation) of several clauses on $N$ variables,
and each clause is a disjunction (an OR operation) of **one or several** variables or negated variables:
>
> $$clause(x) = \bigvee_k y_{k},\text{ where }y_{k} =\text{ either }x_j\text{ or }\neg x_j\text{ for some }j \in \{0, \dots, N-1\}$$
>
> For example, one of the possible clauses on two variables is:
>
> $$clause(x) = x_0 \vee \neg x_1$$

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. a qubit in an arbitrary state $|y\rangle$ (target qubit)

  3. a 1-dimensional array of tuples `clause` which describes one clause of a SAT problem instance $clause(x)$.

`clause` is an array of one or more tuples, each of them describing one component of the clause.

Each tuple is an `(Int, Bool)` pair:

* the first element is the index $j$ of the variable $x_j$, 
* the second element is `true` if the variable is included as itself ($x_j$) and `false` if it is included as a negation ($\neg x_j$).

**Example:**

* The clause $x_0 \vee \neg x_1$ can be represented as `[[(0, true), (1, false)]]`.

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

### Solution

This task involves evaluating a clause which is a disjunction (OR) of negated and non-negated variables encoded in the `queryRegister` . This task can be solved in 2 steps using a similar technique as we saw in Task 1.4 .  
1. We first flip the qubits which are negated in $clause$ (and later undo this operation). 
2. We use the $U_{or}$ unitary to calculate $|y\oplus f(x)\rangle$.

We first conditionally apply a $X$ gate on $|x_j\rangle$ if and only if the clause has a term of the form `(j,false)`.
Next we find `clause_qubits` - qubits which are a negated or non-negated variable in the clause. 
We then apply the $U_{or}$ gate on `clause_qubits` and `target`.

In [ ]:
function getClauseQubits(queryRegister : Qubit[], clause: (Int,Bool)[]) : Qubit[] {
    // Find all qubits which are variables in clause
    mutable clause_qubits = new Qubit[0];
    for((index, positive) in clause) {
        set clause_qubits += [queryRegister[index]];
    }
    return clause_qubits;
}

In [ ]:
%kata T15_Oracle_SATClause_Test 

open Microsoft.Quantum.Arrays;
    
operation Oracle_SATClause (queryRegister : Qubit[], target : Qubit, clause : (Int, Bool)[]) : Unit is Adj {
    // Flip all Qubits which are in negated form in the SAT Clause.
    within{
        for((index, positive) in clause) {
            if (not positive) {
                X(queryRegister[index]);
            }
        }
    }
    apply{
        let clause_qubits = getClauseQubits(queryRegister,clause); // Find All Qubits which are variables in Clause
        Oracle_Or(clause_qubits,target); // Disjuction of variables in clause.
    }
}

We can also use certain Q# library functions and constructs to make our code shorter.
The functions 
[`Fst`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.canon.fst) and
[`Snd`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.canon.snd) 
will be helpful to extract the First and Second Element of the `Pair` respectively. 
The `Fst(term)` would give us the index $j$ for variable $x_j$ and `Snd(term)` would explain if the variable was negated or not. 

We can use 
[`Mapped`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.arrays.mapped)
[`Subarray`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.arrays.subarray) to find the `clause_qubits`.
Given an `array` and a `function` that is defined for the elements of the array, `Mapped` returns a new array that consists of the images of the original array under the function. While `Subarray` takes an `array` and a list of locations and produces a new array formed from the elements of the original array that match the given locations.


In [ ]:
open Microsoft.Quantum.Arrays;
    
operation Oracle_SATClause_2 (queryRegister : Qubit[], target : Qubit, clause : (Int, Bool)[]) : Unit is Adj {
    
    // Flip all Qubits which are in negated form in the SAT Clause.
    within{
        for(term in clause) {
            (Snd(term)? I | X)(queryRegister[Fst(term)]);// If Snd(term) is false then apply X gate otherwise apply I
        }
    }
    apply{
        let indexList = Mapped(Fst<Int,Bool>,clause); // Get Location of all variables in clause
        let clause_qubits = Subarray(indexList,queryRegister); // Find all qubits which are part of clause
        Oracle_Or(clause_qubits,target); // Disjuction of variables in clause.
    }
}

[Return to Task 1.5 of the Solving SAT with Grover's Kata.](./SolveSATWithGrover.ipynb#Task-1.5.-Evaluate-one-clause-of-a-SAT-formula)

### Task 1.6. k-SAT problem oracle

> For k-SAT problems, $f(x)$ is represented as a conjunction (an AND operation) of $M$ clauses on $N$ variables,
and each clause is a disjunction (an OR operation) of **one or several** variables or negated variables:
>
> $$f(x) = \bigwedge_i \big(\bigvee_k y_{ik} \big),\text{ where }y_{ik} =\text{ either }x_j\text{ or }\neg x_j\text{ for some }j \in \{0, \dots, N-1\}$$

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. a qubit in an arbitrary state $|y\rangle$ (target qubit)

  3. a 2-dimensional array of tuples `problem` which describes the k-SAT problem instance $f(x)$.

$i$-th element of `problem` describes the $i$-th clause of $f(x)$; 
it is an array of one or more tuples, each of them describing one component of the clause.

Each tuple is an `(Int, Bool)` pair:

* the first element is the index $j$ of the variable $x_j$, 
* the second element is `true` if the variable is included as itself ($x_j$) and `false` if it is included as a negation ($\neg x_j$)

**Example:**

A more general case of the OR oracle for 3 variables $f(x) = (x_0 \vee x_1 \vee x_2)$ can be represented as `[[(0, true), (1, true), (2, true)]]`.

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

### Solution

This task consists of conjunctions of multiple clause evaluations. Each clause of the form $\bigvee_k y_{ik}$ can be evaluated using the same process as Task 1.5. The answer of these clauses must be stored (temporarily) in freshly allocated qubits. Then the conjunction of these results can be encoded by flipping the `target` qubit.    

Let the number of clauses be $m$, then the steps for implementing k-SAT Oracle would be:
1. Allocate $m$ qubits which are all in state $|0\rangle$.
2. Evaluate Each Clause $clause_i(x)$ using the `operation` created in Task 1.5 such that $|x\rangle|0_i\rangle \rightarrow |X\rangle|clause_i(x)\rangle$
3. Further evaluate $\bigwedge_{i=0}^{n-1}clause_i(x)$ using $U_{and}$ implemented in Task 1.1
4. Undo Step 2 to restore the qubits back into state $|0^{m}\rangle$

We can again use the `within{...}` and `apply{...}` Q# language constructs to perform steps 2,3 and 4.

In [ ]:
%kata T16_Oracle_SAT_Test 

operation Oracle_SAT (queryRegister : Qubit[], target : Qubit, problem : (Int, Bool)[][]) : Unit is Adj {
    using (ancillaRegister = Qubit[Length(problem)]) {
        // Compute clauses all 
        within {
            for (i in 0..Length(problem)-1){
                Oracle_SATClause(queryRegister,ancillaRegister[i],problem[i]);
            }
        }
        // evaluate the overall formula as an AND oracle 
        apply {
            Oracle_And(ancillaRegister, target); // Conjuction of All Clauses
        }
    }
}

[Return to Task 1.6 of the Solving SAT with Grover's Kata.](./SolveSATWithGrover.ipynb#Task-1.6.-k-SAT-problem-oracle)

## Part II. Oracles for exactly-1 3-SAT problem

Exactly-1 3-SAT problem (also known as "one-in-three 3-SAT") is a variant of a general 3-SAT problem.
It has a structure similar to a 3-SAT problem, but each clause must have *exactly one* true literal, 
while in a normal 3-SAT problem each clause must have *at least one* true literal.

### Task 2.1. "Exactly one $|1\rangle$" oracle

**Inputs:** 

  1. 3 qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. a qubit in an arbitrary state $|y\rangle$ (target qubit)

**Goal:** 

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
where $f(x) = 1$ if exactly one bit of $x$ is in the $|1\rangle$ state, and $0$ otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

### Solution

Consider the set of all bitstrings of length $n$ which have only one bit of $x$ in state $1$.   
This set of bitstrings is $S_{One1}=\{00...01, 00...10, ..., 00..1..00, ..., 01...00, 10...00\}$.   
These bitstrings have the equivalent binary representation as $ S_{One1}=\{1,2,4,..2^i,..,2^{n-1}\} = \{2^k: 1<k<n-1\}$.   
Hence if $|x\rangle \in S_{One1}$ then we flip $|y\rangle$. We can represent this as the Unitary $U_{One1}$.  

$$U_{One1} = \sum_{i\in S_{One1}}(|i\rangle\langle i|\otimes X) + \sum_{j \in \{0,1\}^n\setminus S_{One1}}(|j\rangle\langle j|\otimes I) $$
The unitary $U_{One1}$ can be implemented using $n$ Controlled NOT. 
Each Controlled NOT has `queryRegister` as control. The `target` register is flipped if and only if the control is in a particular state of the form $2^k$.

We can use the Q# Library Function [`ControlledOnInt`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.canon.controlledonint) to implement these Controlled NOT gates. 

In [ ]:
%kata T21_Oracle_Exactly1One_Test 

operation Oracle_Exactly1One (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    for(i in 0..Length(queryRegister)-1){
        (ControlledOnInt(2^i,X))(queryRegister,target);// Flip target register if qubit is power of 2
    }
}

[Return to Task 2.1 of the Solving SAT with Grover's Kata.](./SolveSATWithGrover.ipynb#Task-2.1.-"Exactly-one-$|1\rangle$"-oracle)

### Task 2.2. "Exactly-1 3-SAT" oracle

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. a qubit in an arbitrary state $|y\rangle$ (target qubit)

  3. a 2-dimensional array of tuples `problem` which describes the 3-SAT problem instance $f(x)$.

`problem` describes the problem instance in the same format as in task 1.6;
each clause of the formula is guaranteed to have exactly 3 terms.

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

**Example:**

An instance of the problem $f(x) = (x_0 \vee x_1 \vee x_2)$ can be represented as `[[(0, true), (1, true), (2, true)]]`,
and its solutions will be `(true, false, false)`, `(false, true, false)` and `(false, false, true)`, 
but none of the variable assignments in which more than one variable is true, which are solutions for the general SAT problem.

### Solution

This task is similar to Task 1.6. It consists of conjunctions of multiple clause evaluations. However now each clause of the form $\bigvee_k y_{ik}$ must be evaluated under the *Exactly-1 One* policy. Task 1.5 amounted to a `apply Oracle_Or within` conditionally flipped `queryRegister`. Now we have an additional condition that $|x\rangle \in S_{One1}$ i.e. $|x\rangle$ satisfy the conditions of `Oracle_Exactly1One`. Note that $\forall x \in S_{One1}$ $x$ will always satisfy the `Oracle_Or` conditions. Hence we can subsume the former more general oracle under the latter specific one. Thus clause evaluation will be same as Task 1.5 with the exception  of `Oracle_Exactly1One` in place of `Oracle_Or` we would use .

The answer of these clauses will be stored (temporarily) in freshly allocated qubits. Then the conjunction of these results can be encoded by flipping the `target` qubit.

Let the number of clauses be $m$, then the steps for implementing Exactly-1 3-SAT Oracle would be:
1. Allocate $m$ qubits which are all in state $|0\rangle$.
2. Evaluate Each Clause $clause_i(x)$ using the above modification of `operation` created in Task 1.5 such that $|x\rangle|0_i\rangle \rightarrow |X\rangle|clause_i(x)\rangle$
3. Further evaluate $\bigwedge_{i=0}^{n-1}clause_i(x)$ using $U_{and}$ implemented in Task 1.1
4. Undo Step 2 to restore the qubits back into state $|0^{m}\rangle$

We can again use the `within{...}` and `apply{...}` Q# language constructs to perform steps 2,3 and 4.

In [ ]:
open Microsoft.Quantum.Arrays;
    
operation Oracle_Exactly1OneSATClause (queryRegister : Qubit[], target : Qubit, clause : (Int, Bool)[]) : Unit is Adj {
    
    // Flip the Qubits which are negated.
    // Flip all Qubits which are in negated form in the SAT Clause.
    within{
        for((index, positive) in clause) {
            if (not positive) {
                X(queryRegister[index]);
            }
        }
    }
    apply{
        let clause_qubits = getClauseQubits(queryRegister,clause); // Find All Qubits which are variables in Clause
        Oracle_Exactly1One(clause_qubits,target); // Check if Exactly One Criteria is satisified 
    }
}

In [ ]:
%kata T22_Oracle_Exactly1SAT_Test 

operation Oracle_Exactly1_3SAT (queryRegister : Qubit[], target : Qubit, problem : (Int, Bool)[][]) : Unit is Adj {
    using (ancillaRegister = Qubit[Length(problem)]) {
            // Compute clauses, evaluate the overall formula as an AND oracle (can use reference depending on the implementation) and uncompute
        within {
            for (i in 0..Length(problem)-1){
                Oracle_Exactly1OneSATClause(queryRegister,ancillaRegister[i],problem[i]);
            }
        }
        apply {
            Oracle_And(ancillaRegister, target); // Conjuction of All Clauses
        }
    }
}

[Return to Task 2.2 of the Solving SAT with Grover's Kata.](./SolveSATWithGrover.ipynb#Task-2.2.-"Exactly-1-3-SAT"-oracle)

## Part III. Using Grover's algorithm for problems with multiple solutions

### Task 3.1. Using Grover's algorithm

**Goal:**

Implement Grover's algorithm and use it to find solutions to SAT instances from parts I and II. 

> If you want to learn the Grover's algorithm itself, try doing [GroversAlgorithm kata](./../GroversAlgorithm/GroversAlgorithm.ipynb) first.

> This is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_GroversSearch_Algorithm` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_GroversSearch_Algorithm`).

> Note that this task relies on your implementations of the previous tasks. If you are getting the "No variable with that name exists." error, you might have to execute previous code cells before retrying this task.


### Solution

This task can be solved  following steps:
1. Grover's Algorithm requires a phase oracle and hence we first must write Q# code for an `Oracle_Converter`. 
This converts a marking (or flipping) oracle to a phase oracle.
2. Write an operation `GroversSearch_Algorithm` which takes `register`,`numIterations` and `markingOracle` as inputs. 
This operation runs the Grover's Algorithm Iteration on the `register` `numIterations` times. 
This operation finally outputs a state which satisfies the conditions of the marking oracle with high probability.
3. Write the driver operation `Run_GroversSearch_Algorithm` which runs the `GroversSearch_Algorithm` multiples times till it outputs a solutions state. 

The parameters defining size of register `N` and `oracle` are upto you!

Try Experimenting with the Number of Iterations in the driver code.  
Remember that for an oracle which has $k << N, N=2^n$ marked states, the optimal `numIterations`$ \approx \frac{\pi}{4}\sqrt{\frac{N}{k}}$ 

All three pieces of code can be created in a manner specified by [GroversAlgorithm Kata](../GroversAlgorithm/GroversAlgorithm.ipynb).

In [ ]:
operation Oracle_Converter (markingOracle : ((Qubit[], Qubit) => Unit is Adj), register : Qubit[]) : Unit is Adj {
    using (target = Qubit()) {
        // Put the target into the |-⟩ state and later revert the state
        within{ 
            X(target);
            H(target); 
        }
        // Apply the marking oracle; since the target is in the |-⟩ state,
        // flipping the target if the register satisfies the oracle condition will apply a -1 factor to the state
        apply{ 
            markingOracle(register, target);
        }
    }
}

In [ ]:
open Microsoft.Quantum.Arrays;

operation GroversSearch_Algorithm (register: Qubit[], oracle: ((Qubit[], Qubit) => Unit is Adj), numIterations: Int): Unit {
    let phaseOracle = Oracle_Converter(oracle,_);
    ApplyToEach(H, register); // Create Uniform Superposition of all states

    for (i in 1 .. numIterations) {
        phaseOracle(register);
        within{
            ApplyToEachA(H, register);
            ApplyToEachA(X, register);
        } 
        apply{
            Controlled Z(Most(register), Tail(register));
        }
    }
}

In [ ]:
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;

operation Run_GroversSearch_Algorithm () : Unit {
    let oracle = Oracle_Xor; // Try with different Oracles
    let N = 4; // Problem Size
    let numIterations = 10; // Experiment with this Number
    using ((register, output) = (Qubit[N], Qubit())) {
        Message($"Trying search with {numIterations} iterations");
        GroversSearch_Algorithm(register, oracle, numIterations); // Do numIterations of Grovers Search Algorithm
        let res = MultiM(register); // Measure the register
        oracle(register, output); // apply oracle to the register after measurement to check the result is correct. 
        
        if (MResetZ(output) == One) { // Check if output is One to confirm correctness
            let answer = ResultArrayAsBoolArray(res);
            Message($"Found Answer: {answer}");
        } else {
            Message("Failed to find an answer. Try again!");
        }
        ResetAll(register);        
    }
}

In [ ]:
%simulate Run_GroversSearch_Algorithm

[Return to Task 3.1 of the Solving SAT with Grover's Kata.](./SolveSATWithGrover.ipynb#Task-3.1.-Using-Grover's-algorithm)

### Task 3.2. Universal implementation of Grover's algorithm

**Inputs:**

  1. the number of qubits N,

  2. a marking oracle which implements a boolean expression, similar to the oracles from part I.

**Output:**

An array of N boolean values which satisfy the expression implemented by the oracle 
(i.e., any basis state marked by the oracle).

> Note that the similar task in the GroversAlgorithm kata required you to implement Grover's algorithm 
in a way that would be robust to accidental failures, but you knew the optimal number of iterations 
(the number that minimized the probability of such failure).
> 
> In this task you also need to make your implementation robust to not knowing the optimal number of iterations.

### Solution

This is similar to Task 3.1 but we don't know the `oracle` and hence have no way of predicting the optimal number of transitions. Instead we try the series $\{1,2,4,8,...\}$ as the number of transitions. Even if none of these transitions are optimal we will still eventually get the correct answer with high probability. 

We use the RUS or `repeat{...} until(...) fixup{...}` constructs of Q# language to try these series of iterations `until` we find an `answer`. The `repeat` consists of performing the `GroversSearch_Algorithm` operations, measuring the result and checking if the result is correct. The `until` construct checks if we found a correct or answer or reached a time-out condition. The `fixup` involves doubling the number of iterations in order to try again.

In [ ]:
%kata T32_UniversalGroversAlgorithm_Test 

open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;

operation UniversalGroversAlgorithm (N : Int, oracle : ((Qubit[], Qubit) => Unit is Adj)) : Bool[] {
    // Since we are unaware of the optimal number of iterations, we try different numbers of iterations.
    // This way we should eventually get a high enough success probability.

    mutable answer = new Bool[N];
    using ((register, output) = (Qubit[N], Qubit())) {
        mutable correct = false;
        mutable iter = 1;
        repeat {
            Message($"Trying search with {iter} iterations");
            GroversSearch_Algorithm(register, oracle, iter);
            let res = MultiM(register);
            
            oracle(register, output); // apply oracle to the register after measurement to check the result is correct. 
            if (MResetZ(output) == One) {
                set correct = true;
                set answer = ResultArrayAsBoolArray(res);
            }
            ResetAll(register);
        } until (correct or iter > 100)  // the fail-safe to avoid going into an infinite loop
        fixup {
            set iter *= 2;
        }
        if (not correct) {
            fail "Failed to find an answer";
        }
    }
    Message($"{answer}");
    return answer;
}


[Return to Task 3.2 of the Solving SAT with Grover's Kata.](./SolveSATWithGrover.ipynb#Task-3.2.-Universal-implementation-of-Grover's-algorithm)